# Fetch Binance Data

In [1]:
import requests
import pandas as pd
from datetime import datetime
from tqdm import tqdm

BINANCE_CANDLE_COLUMNS = ['opentime', 'openprice', 'highprice', 'lowprice', 'closeprice', 'volume', 'closetime',
                          'quotevolume', 'trades', 'taker_buy_volume', 'taker_buy_quote', 'unused']

def binance_recursive_fetch_2(coins, interval, starttime, endtime, data_type='spot'):
    all_coins_result = {}
    data_list = []
    call_dict = {}
    
    for coin in tqdm(coins):
        result_list = []
        current_time = starttime
        call = 0
        timestamps = []
        
        while current_time < endtime:
            if ((int((endtime - current_time) / (1000 * 60))) + 1) >= 1000:
                limit = 1000
            else:
                limit = int((endtime - current_time) / (1000 * 60) + 1)
            
            if data_type == 'spot':
                url = (f'https://api.binance.com/api/v3/klines'
                       f'?symbol={coin}USDT'
                       f'&startTime={str(current_time)}'
                       f'&interval={interval}'
                       f'&limit={str(limit)}')
            elif data_type == 'futures':
                url = (f'https://fapi.binance.com/fapi/v1/klines'
                       f'?symbol={coin}USDT'
                       f'&startTime={str(current_time)}'
                       f'&interval={interval}'
                       f'&limit={str(limit)}')
            
            result_list += requests.get(url).json()
            
            if result_list:
                # Update current_time with the timestamp of the last data point fetched, plus 1 minute (60000 ms)
                current_time = result_list[-1][0] + 60000
                timestamps.append(current_time)
                call += 1
                
                # Check if the last fetched timestamp is greater than or equal to endtime
                if current_time >= endtime:
                    print(f"Reached endtime at {datetime.fromtimestamp(current_time / 1000).strftime('%Y-%m-%d %H:%M:%S')}. Stopping fetch.")
                    break
                
                print((datetime.fromtimestamp(current_time / 1000).strftime('%Y-%m-%d %H:%M:%S')) + 
                      f' status : {current_time < endtime}, time : {current_time}, limit : {call * 2}')
            
            # Ensure there's no continuous fetching of the same timestamp
            if len(timestamps) > 1 and timestamps[-1] == timestamps[-2]:
                print("Duplicate timestamp detected. Stopping fetch.")
                break
            
            # Sleep if needed to avoid rate limiting (adjust based on your rate limit)
            # time.sleep(0.1)  # Uncomment if needed
            
        current_df = pd.DataFrame(result_list, columns=BINANCE_CANDLE_COLUMNS)
        current_df['coin'] = coin
        current_df = current_df[['coin'] + BINANCE_CANDLE_COLUMNS]
        current_df = current_df.values.tolist()
        
        data_list += current_df
        call_dict.update({coin: call})
    
    return {'data': data_list, 'call': call_dict}

# Set endtime to the current time (today)
endtime = int(datetime.utcnow().timestamp() * 1000)

/tmp/ipykernel_3516249/2566178532.py:74: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  endtime = int(datetime.utcnow().timestamp() * 1000)


In [2]:
# Fetch NEAR USDT
sample_spot = binance_recursive_fetch_2(
    ['BTC'],
    '1m',
    starttime=int(pd.to_datetime('2025-01-01 00:00', utc=True).timestamp() * 1000),
    endtime=int(pd.to_datetime('2025-03-01 00:00', utc=True).timestamp() * 1000),
    data_type='futures'  # Fetch spot data
)

# Define the column names for the DataFrame based on the Binance API response structure
columns = ['coin', 'opentime', 'open', 'high', 'low', 'close', 'volume', 'closetime', 
           'quotevolume', 'trades', 'taker_buy_volume', 'taker_buy_quote', 'unused']

# Convert the list of data into a DataFrame
df = pd.DataFrame(sample_spot['data'], columns=columns)

df = df[['opentime', 'open', 'high', 'low', 'close']]

df

  0%|          | 0/1 [00:00<?, ?it/s]

2025-01-01 16:40:00 status : True, time : 1735749600000, limit : 2
2025-01-02 09:20:00 status : True, time : 1735809600000, limit : 4
2025-01-03 02:00:00 status : True, time : 1735869600000, limit : 6
2025-01-03 18:40:00 status : True, time : 1735929600000, limit : 8
2025-01-04 11:20:00 status : True, time : 1735989600000, limit : 10
2025-01-05 04:00:00 status : True, time : 1736049600000, limit : 12
2025-01-05 20:40:00 status : True, time : 1736109600000, limit : 14
2025-01-06 13:20:00 status : True, time : 1736169600000, limit : 16
2025-01-07 06:00:00 status : True, time : 1736229600000, limit : 18
2025-01-07 22:40:00 status : True, time : 1736289600000, limit : 20
2025-01-08 15:20:00 status : True, time : 1736349600000, limit : 22
2025-01-09 08:00:00 status : True, time : 1736409600000, limit : 24
2025-01-10 00:40:00 status : True, time : 1736469600000, limit : 26
2025-01-10 17:20:00 status : True, time : 1736529600000, limit : 28
2025-01-11 10:00:00 status : True, time : 1736589600

100%|██████████| 1/1 [00:09<00:00,  9.82s/it]


,opentime,open,high,low,close
0,1735689600000,93548.80,93599.90,93514.20,93599.90
1,1735689660000,93599.90,93637.70,93577.60,93637.70
2,1735689720000,93637.70,93690.00,93614.20,93688.50
3,1735689780000,93688.50,93688.50,93626.40,93664.60
4,1735689840000,93664.50,93668.30,93626.30,93648.40
...,...,...,...,...,...
84956,1740786960000,84385.00,84387.70,84377.50,84385.90
84957,1740787020000,84385.90,84386.00,84338.90,84343.20
84958,1740787080000,84343.20,84368.10,84343.10,84368.10
84959,1740787140000,84368.00,84375.60,84292.80,84299.60


In [3]:
import polars as pl

dfs = pl.from_pandas(df)

dfs = dfs.with_columns([
    dfs["open"].cast(pl.Float64),
    dfs["high"].cast(pl.Float64),
    dfs["low"].cast(pl.Float64),
    dfs["close"].cast(pl.Float64)
])

dfs

opentime,open,high,low,close
i64,f64,f64,f64,f64
1735689600000,93548.8,93599.9,93514.2,93599.9
1735689660000,93599.9,93637.7,93577.6,93637.7
1735689720000,93637.7,93690.0,93614.2,93688.5
1735689780000,93688.5,93688.5,93626.4,93664.6
1735689840000,93664.5,93668.3,93626.3,93648.4
…,…,…,…,…
1740786960000,84385.0,84387.7,84377.5,84385.9
1740787020000,84385.9,84386.0,84338.9,84343.2
1740787080000,84343.2,84368.1,84343.1,84368.1


# Load Full Data

In [167]:
import polars as pl
from pathlib import Path

def read_aggregated_files(base_path, symbol, interval, year=2024):
    """
    Reads and concatenates aggregated trade data files for a given symbol and interval.

    Parameters:
    - base_path: The base directory where the data is stored.
    - symbol: The trading symbol (e.g., 'BTCUSDT').
    - interval: The aggregation interval (e.g., '15s', '20s', '25s', '30s').
    - year: The year of the data (default is 2024).

    Returns:
    - A concatenated Polars DataFrame containing all the data with consistent Float64 column types.
    """
    # Construct the directory path
    data_dir = Path(base_path) / f"{symbol}_perps" / f"agg_{interval}"
    
    # Generate the list of file paths
    files = [
        data_dir / f"bybit_{symbol.lower()}_aggtrades_{year}-{month:02d}_aggregated_{interval}.parquet"
        for month in range(1, 13)
    ]
    
    # Read and convert all files to Float64 before concatenation
    dfs = []
    for file in files:
        if file.exists():
            df = pl.read_parquet(file)
            df = df.with_columns([pl.col(col).cast(pl.Float64) for col in df.columns])  # Corrected casting
            dfs.append(df)
    
    # Concatenate vertically
    return pl.concat(dfs) if dfs else pl.DataFrame()

# Example usage
base_path = "/home/ubuntu/Rheza/data/bybit_trades_data"
symbol = "BTCUSDT"
interval = "60s"

dfs = read_aggregated_files(base_path, symbol, interval)
dfs = dfs.filter(dfs["interval"] == 0)
dfs = dfs.drop(["interval","buy_size","buy_volume","sell_size","sell_volume"])
dfs.head(5)

year,month,day,hour,minute,open,high,low,close
f64,f64,f64,f64,f64,f64,f64,f64,f64
2024.0,1.0,1.0,0.0,0.0,42324.9,42349.9,42300.2,42346.8
2024.0,1.0,1.0,0.0,1.0,42346.8,42373.9,42346.8,42363.5
2024.0,1.0,1.0,0.0,2.0,42363.5,42379.2,42362.5,42369.7
2024.0,1.0,1.0,0.0,3.0,42369.7,42423.0,42369.7,42423.0
2024.0,1.0,1.0,0.0,4.0,42423.0,42462.0,42423.0,42452.1


# Features

In [5]:
import polars as pl

# Previous 100 rows max close & min close
dfs_featured = dfs.with_columns(
    pl.col("close").rolling_max(window_size=50).alias("prev_100_max_close"),
    pl.col("close").rolling_min(window_size=50).alias("prev_100_min_close")
)

# Next row close (-1, -2, -3)
dfs_featured = dfs_featured.with_columns(
    pl.col("close").shift(-1).alias("next_1_close"),
    pl.col("close").shift(-2).alias("next_2_close"),
    pl.col("close").shift(-3).alias("next_3_close")
)

dfs_featured = dfs_featured.drop_nulls()

# Display result
dfs_featured

opentime,open,high,low,close,prev_100_max_close,prev_100_min_close,next_1_close,next_2_close,next_3_close
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1704070140000,0.08996,0.08996,0.08993,0.08995,0.09007,0.08961,0.08995,0.08989,0.08987
1704070200000,0.08995,0.08997,0.08995,0.08995,0.09007,0.08967,0.08989,0.08987,0.0899
1704070260000,0.08995,0.08995,0.08989,0.08989,0.09007,0.08967,0.08987,0.0899,0.08991
1704070320000,0.08988,0.08991,0.08987,0.08987,0.09007,0.08978,0.0899,0.08991,0.08992
1704070380000,0.08988,0.08991,0.08987,0.0899,0.09007,0.08978,0.08991,0.08992,0.08996
…,…,…,…,…,…,…,…,…,…
1735689180000,0.31652,0.31656,0.31626,0.31626,0.31682,0.31524,0.3163,0.31642,0.31622
1735689240000,0.31626,0.31639,0.31626,0.3163,0.31682,0.31524,0.31642,0.31622,0.3159
1735689300000,0.3163,0.31643,0.31617,0.31642,0.31682,0.31524,0.31622,0.3159,0.31597


# Flagging Signal

In [6]:
# First Break Out
dfs_flagged = dfs_featured.with_columns(
    pl.when(pl.col("close") >= pl.col("prev_100_max_close"))
    .then(1)
    .when(pl.col("close") <= pl.col("prev_100_min_close"))
    .then(-1)
    .otherwise(0)
    .alias("prev_100_hit")
)

dfs_flagged = dfs_flagged.with_columns(
    pl.when(
        (pl.col("next_1_close") >= pl.col("prev_100_max_close")) &
        (pl.col("next_2_close") >= pl.col("prev_100_max_close")) &
        (pl.col("next_3_close") >= pl.col("prev_100_max_close"))
    )
    .then(1)
    .when(
        (pl.col("next_1_close") <= pl.col("prev_100_min_close")) &
        (pl.col("next_2_close") <= pl.col("prev_100_min_close")) &
        (pl.col("next_3_close") <= pl.col("prev_100_min_close"))
    )
    .then(-1)
    .otherwise(0)
    .alias("3_candles_hit")
)

# Get the next 20 rows hit
dfs_flagged = dfs_flagged.with_columns(
    pl.col("prev_100_hit").rolling_max(window_size=50).shift(-51).alias("next_20_hit_up"),
    pl.col("prev_100_hit").rolling_min(window_size=50).shift(-51).alias("next_20_hit_down")
)

dfs_flagged = dfs_flagged.with_columns(
    pl.col("next_20_hit_up").fill_null(0),
    pl.col("next_20_hit_down").fill_null(0)
)

# Fast Reversal Breakout
dfs_flagged = dfs_flagged.with_columns(
    pl.when(
        (pl.col("next_20_hit_up") == 1)
    )
    .then(1)
    .when(
        (pl.col("next_20_hit_down") == -1)
    )
    .then(-1)
    .otherwise(0)
    .alias("fast_reversal_breakout")
)

# Display result
dfs_flagged

opentime,open,high,low,close,prev_100_max_close,prev_100_min_close,next_1_close,next_2_close,next_3_close,prev_100_hit,3_candles_hit,next_20_hit_up,next_20_hit_down,fast_reversal_breakout
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32,i32,i32,i32,i32
1704070140000,0.08996,0.08996,0.08993,0.08995,0.09007,0.08961,0.08995,0.08989,0.08987,0,0,1,-1,1
1704070200000,0.08995,0.08997,0.08995,0.08995,0.09007,0.08967,0.08989,0.08987,0.0899,0,0,1,-1,1
1704070260000,0.08995,0.08995,0.08989,0.08989,0.09007,0.08967,0.08987,0.0899,0.08991,0,0,1,-1,1
1704070320000,0.08988,0.08991,0.08987,0.08987,0.09007,0.08978,0.0899,0.08991,0.08992,0,0,1,-1,1
1704070380000,0.08988,0.08991,0.08987,0.0899,0.09007,0.08978,0.08991,0.08992,0.08996,0,0,1,-1,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1735689180000,0.31652,0.31656,0.31626,0.31626,0.31682,0.31524,0.3163,0.31642,0.31622,0,0,0,0,0
1735689240000,0.31626,0.31639,0.31626,0.3163,0.31682,0.31524,0.31642,0.31622,0.3159,0,0,0,0,0
1735689300000,0.3163,0.31643,0.31617,0.31642,0.31682,0.31524,0.31622,0.3159,0.31597,0,0,0,0,0


# Determining Sequence Approach 1

In [22]:
import polars as pl

# Initialize sequence column with None
dfs_sequenced = dfs_flagged.with_columns([
    pl.lit(None, dtype=pl.Utf8).alias("sequence"),
    pl.lit(None, dtype=pl.Float64).alias("Upper Long Zone"),
    pl.lit(None, dtype=pl.Float64).alias("Lower Long Zone"),
    pl.lit(None, dtype=pl.Float64).alias("Upper Short Zone"),
    pl.lit(None, dtype=pl.Float64).alias("Lower Short Zone"),
])

# Define a function to iterate and determine the sequence for each row
def track_sequence(df):
    sequence = []
    upper_long_zone = []
    lower_long_zone = []
    upper_short_zone = []
    lower_short_zone = []
    
    prev_seq = None
    prev_upper_long = None
    prev_lower_long = None
    prev_upper_short = None
    prev_lower_short = None

    for row in df.iter_rows(named=True):
        prev_100_hit = row["prev_100_hit"]
        three_candles_hit = row["3_candles_hit"]
        fast_reversal_breakout = row["fast_reversal_breakout"]
        
        prev_100_min_close = row["prev_100_min_close"]
        prev_100_max_close = row["prev_100_max_close"]
        next_1_close = row["next_1_close"]
        next_2_close = row["next_2_close"]
        next_3_close = row["next_3_close"]

        # Default to previous sequence unless a condition changes it
        seq = prev_seq

        if prev_seq is None:  # If there is no sequence yet or sequence reset
            # Condition 1: Unconfirmed Break Up
            if prev_100_hit == -1 and three_candles_hit == -1 and fast_reversal_breakout == 1:
                seq = "Unconfirmed Break Up"
                prev_upper_long = prev_100_min_close
                prev_lower_long = min(next_1_close, next_2_close, next_3_close)

            # Condition 2: Unconfirmed Break Down
            elif prev_100_hit == 1 and three_candles_hit == 1 and fast_reversal_breakout == -1:
                seq = "Unconfirmed Break Down"
                prev_upper_short = max(next_1_close, next_2_close, next_3_close)
                prev_lower_short = prev_100_max_close

        # Condition 3: After "Unconfirmed Break Up"
        elif prev_seq == "Unconfirmed Break Up":
            if prev_100_hit == 1 and three_candles_hit == 1:
                seq = "Confirmed Break Up"
            elif prev_100_hit == -1 and three_candles_hit == -1:
                seq = "Unconfirmed Break Down"
                prev_upper_short = max(next_1_close, next_2_close, next_3_close)
                prev_lower_short = prev_100_max_close

        # Condition 4: After "Unconfirmed Break Down"
        elif prev_seq == "Unconfirmed Break Down":
            if prev_100_hit == -1 and three_candles_hit == -1:
                seq = "Confirmed Break Down"
            elif prev_100_hit == 1 and three_candles_hit == 1:
                seq = "Unconfirmed Break Up"
                prev_upper_long = prev_100_min_close
                prev_lower_long = min(next_1_close, next_2_close, next_3_close)

        # Condition 5: After "Confirmed Break Up"
        elif prev_seq == "Confirmed Break Up":
            max_3_candles_close = max(next_1_close, next_2_close, next_3_close)
            if prev_upper_long is not None:
                x_percent = (prev_upper_long - max_3_candles_close) / prev_upper_long
                if prev_100_min_close < prev_lower_long * (1 - x_percent):
                    seq = None  # Reset sequence

        # Condition 6: After "Confirmed Break Down"
        elif prev_seq == "Confirmed Break Down":
            min_3_candles_close = min(next_1_close, next_2_close, next_3_close)
            if prev_lower_short is not None:
                x_percent = (prev_lower_short - min_3_candles_close) / prev_lower_short
                if prev_100_max_close > prev_upper_short * (1 + x_percent):
                    seq = None  # Reset sequence

        # Assigning stored values accordingly
        if seq in ["Unconfirmed Break Up", "Confirmed Break Up"]:
            upper_long_zone.append(prev_upper_long)
            lower_long_zone.append(prev_lower_long)
            upper_short_zone.append(None)
            lower_short_zone.append(None)
        elif seq in ["Unconfirmed Break Down", "Confirmed Break Down"]:
            upper_long_zone.append(None)
            lower_long_zone.append(None)
            upper_short_zone.append(prev_upper_short)
            lower_short_zone.append(prev_lower_short)
        else:
            upper_long_zone.append(None)
            lower_long_zone.append(None)
            upper_short_zone.append(None)
            lower_short_zone.append(None)

        # Append sequence and update previous sequence tracker
        sequence.append(seq)
        prev_seq = seq  # Carry forward the sequence to the next row

    # Return DataFrame with updated sequence column
    return df.with_columns([
        pl.Series(sequence, dtype=pl.Utf8, strict=True).alias("sequence"),
        pl.Series(upper_long_zone, dtype=pl.Float64, strict=True).alias("Upper Long Zone"),
        pl.Series(lower_long_zone, dtype=pl.Float64, strict=True).alias("Lower Long Zone"),
        pl.Series(upper_short_zone, dtype=pl.Float64, strict=True).alias("Upper Short Zone"),
        pl.Series(lower_short_zone, dtype=pl.Float64, strict=True).alias("Lower Short Zone"),
    ])

# Apply the function
dfs_sequenced = track_sequence(dfs_sequenced)

# Display result
dfs_sequenced

opentime,open,high,low,close,prev_100_max_close,prev_100_min_close,next_1_close,next_2_close,next_3_close,prev_100_hit,3_candles_hit,next_20_hit_up,next_20_hit_down,fast_reversal_breakout,sequence,Upper Long Zone,Lower Long Zone,Upper Short Zone,Lower Short Zone
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32,i32,i32,i32,i32,str,f64,f64,f64,f64
1735695540000,93909.6,93909.8,93885.1,93886.9,94439.5,93485.8,93917.1,93927.1,93914.9,0,0,0,-1,-1,null,null,null,null,null
1735695600000,93886.9,93917.1,93885.0,93917.1,94439.5,93485.8,93927.1,93914.9,93935.9,0,0,0,-1,-1,null,null,null,null,null
1735695660000,93917.1,93951.3,93903.0,93927.1,94439.5,93485.8,93914.9,93935.9,93892.1,0,0,0,-1,-1,null,null,null,null,null
1735695720000,93927.1,93927.2,93914.9,93914.9,94439.5,93485.8,93935.9,93892.1,93907.7,0,0,0,-1,-1,null,null,null,null,null
1735695780000,93914.8,93944.5,93914.8,93935.9,94439.5,93485.8,93892.1,93907.7,93908.3,0,0,0,-1,-1,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1740009120000,96564.3,96564.4,96531.5,96531.6,96786.4,96305.4,96522.4,96525.4,96525.4,0,0,null,null,0,null,null,null,null,null
1740009180000,96531.5,96531.6,96513.6,96522.4,96786.4,96305.4,96525.4,96525.4,96533.9,0,0,null,null,0,null,null,null,null,null
1740009240000,96522.5,96525.5,96513.6,96525.4,96786.4,96305.4,96525.4,96533.9,96604.5,0,0,null,null,0,null,null,null,null,null


# Determining Sequence Approach 2

In [ ]:
import polars as pl

def track_sequence(df):
    
    sequence = []
    upper_long_zone = []
    lower_long_zone = []
    upper_short_zone = []
    lower_short_zone = []
    buy_zone_hit = []
    target_hit = []
    first_sequence = []
    target_prices = []

    prev_seq = None
    prev_upper_long = None
    prev_lower_long = None
    prev_upper_short = None
    prev_lower_short = None

    confirmed_index = None
    prev_target_price = 0  
    prev_buy_zone_low = None
    prev_buy_zone_high = None
    has_hit_buy_zone = False  

    for i, row in enumerate(df.iter_rows(named=True)):
        prev_100_hit = row["prev_100_hit"]
        three_candles_hit = row["3_candles_hit"]
        fast_reversal_breakout = row["fast_reversal_breakout"]
        prev_100_min_close = row["prev_100_min_close"]
        prev_100_max_close = row["prev_100_max_close"]
        next_1_close = row["next_1_close"]
        next_2_close = row["next_2_close"]
        next_3_close = row["next_3_close"]
        low = row["low"]
        high = row["high"]

        buy_zone_hit_flag = 0
        target_hit_flag = 0
        first_seq_flag = 0

        if prev_seq is None:
            if prev_100_hit == -1 and three_candles_hit == -1 and fast_reversal_breakout == 1:
                seq = "Unconfirmed Break Up"
                prev_upper_long = prev_100_min_close
                prev_lower_long = min(next_1_close, next_2_close, next_3_close)
                first_seq_flag = 1
            elif prev_100_hit == 1 and three_candles_hit == 1 and fast_reversal_breakout == -1:
                seq = "Unconfirmed Break Down"
                prev_upper_short = max(next_1_close, next_2_close, next_3_close)
                prev_lower_short = prev_100_max_close
                first_seq_flag = 1
            else:
                seq = None

        elif prev_seq == "Unconfirmed Break Up":
            if prev_100_hit == 1 and three_candles_hit == 1:
                seq = "Confirmed Break Up"
                confirmed_index = i
                prev_target_price = prev_100_max_close * (1 + (prev_100_max_close - prev_100_min_close) / prev_100_min_close)
                prev_buy_zone_low = prev_100_min_close
                prev_buy_zone_high = prev_lower_long
                has_hit_buy_zone = False
                first_seq_flag = 1
            elif prev_100_hit == -1 and three_candles_hit == -1:
                seq = "Unconfirmed Break Down"
                prev_upper_short = max(next_1_close, next_2_close, next_3_close)
                prev_lower_short = prev_100_max_close
                first_seq_flag = 1
            else:
                seq = prev_seq

        elif prev_seq == "Unconfirmed Break Down":
            if prev_100_hit == -1 and three_candles_hit == -1:
                seq = "Confirmed Break Down"
                confirmed_index = i
                prev_target_price = prev_100_min_close * (1 - (prev_100_max_close - prev_100_min_close) / prev_100_max_close)
                prev_buy_zone_high = prev_100_max_close
                prev_buy_zone_low = prev_upper_short
                has_hit_buy_zone = False
                first_seq_flag = 1
            elif prev_100_hit == 1 and three_candles_hit == 1:
                seq = "Unconfirmed Break Up"
                prev_upper_long = prev_100_min_close
                prev_lower_long = min(next_1_close, next_2_close, next_3_close)
                first_seq_flag = 1
            else:
                seq = prev_seq

        elif prev_seq == "Confirmed Break Up":
            if confirmed_index is not None and (i - confirmed_index >= 3000):
                seq = None
            elif prev_buy_zone_low <= low <= prev_buy_zone_high:  
                seq = "Confirmed Break Up"
                if not has_hit_buy_zone:
                    buy_zone_hit_flag = 1
                    has_hit_buy_zone = True
            elif has_hit_buy_zone and prev_target_price > 0 and high >= prev_target_price:  
                target_hit_flag = 1
                seq = None  # Reset sequence after target is hit
            else:
                seq = prev_seq

        elif prev_seq == "Confirmed Break Down":
            if confirmed_index is not None and (i - confirmed_index >= 3000):
                seq = None
            elif prev_buy_zone_low <= high <= prev_buy_zone_high:  
                seq = "Confirmed Break Down"
                if not has_hit_buy_zone:
                    buy_zone_hit_flag = 1
                    has_hit_buy_zone = True
            elif has_hit_buy_zone and prev_target_price > 0 and low <= prev_target_price:  
                target_hit_flag = 1
                seq = None  # Reset sequence after target is hit
            else:
                seq = prev_seq

        else:
            seq = None

        if seq != prev_seq:
            first_seq_flag = 1  

        # Reset everything if the target is hit
        if target_hit_flag == 1:
            prev_seq = None
            prev_upper_long = None
            prev_lower_long = None
            prev_upper_short = None
            prev_lower_short = None
            confirmed_index = None
            prev_target_price = 0
            prev_buy_zone_low = None
            prev_buy_zone_high = None
            has_hit_buy_zone = False
        else:
            prev_seq = seq  

        sequence.append(seq)
        buy_zone_hit.append(buy_zone_hit_flag)
        target_hit.append(target_hit_flag)
        first_sequence.append(first_seq_flag)

        if seq in ["Confirmed Break Up", "Confirmed Break Down"]:
            target_prices.append(prev_target_price)
        else:
            target_prices.append(0)

        if seq in ["Unconfirmed Break Up", "Confirmed Break Up"]:
            upper_long_zone.append(prev_upper_long)
            lower_long_zone.append(prev_lower_long)
            upper_short_zone.append(None)
            lower_short_zone.append(None)
        elif seq in ["Unconfirmed Break Down", "Confirmed Break Down"]:
            upper_long_zone.append(None)
            lower_long_zone.append(None)
            upper_short_zone.append(prev_upper_short)
            lower_short_zone.append(prev_lower_short)
        else:
            upper_long_zone.append(None)
            lower_long_zone.append(None)
            upper_short_zone.append(None)
            lower_short_zone.append(None)

    # Ensure `target_hit` is `0` if `sequence` is `None`
    target_hit = [0 if seq is None else hit for seq, hit in zip(sequence, target_hit)]

    return df.with_columns([
        pl.Series(sequence, dtype=pl.Utf8, strict=True).alias("sequence"),
        pl.Series(upper_long_zone, dtype=pl.Float64, strict=True).alias("Upper Long Zone"),
        pl.Series(lower_long_zone, dtype=pl.Float64, strict=True).alias("Lower Long Zone"),
        pl.Series(upper_short_zone, dtype=pl.Float64, strict=True).alias("Upper Short Zone"),
        pl.Series(lower_short_zone, dtype=pl.Float64, strict=True).alias("Lower Short Zone"),
        pl.Series(buy_zone_hit, dtype=pl.Int8, strict=True).alias("buy_zone_hit"),
        pl.Series(target_hit, dtype=pl.Int8, strict=True).alias("target_hit"),
        pl.Series(first_sequence, dtype=pl.Int8, strict=True).alias("first_sequence"),
        pl.Series(target_prices, dtype=pl.Float64, strict=True).alias("target_price"),
    ])

# Apply function
dfs_sequenced = track_sequence(dfs_flagged)
dfs_sequenced

opentime,open,high,low,close,prev_100_max_close,prev_100_min_close,next_1_close,next_2_close,next_3_close,prev_100_hit,3_candles_hit,next_20_hit_up,next_20_hit_down,fast_reversal_breakout,sequence,Upper Long Zone,Lower Long Zone,Upper Short Zone,Lower Short Zone,buy_zone_hit,target_hit,first_sequence,target_price
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32,i32,i32,i32,i32,str,f64,f64,f64,f64,i8,i8,i8,f64
1735701540000,93837.2,93837.5,93818.6,93818.7,94439.5,93485.8,93777.5,93809.2,93800.0,0,0,0,0,0,null,null,null,null,null,0,0,0,0.0
1735701600000,93818.7,93818.7,93760.6,93777.5,94439.5,93485.8,93809.2,93800.0,93812.7,0,0,0,-1,-1,null,null,null,null,null,0,0,0,0.0
1735701660000,93777.5,93809.3,93736.1,93809.2,94439.5,93485.8,93800.0,93812.7,93808.9,0,0,0,-1,-1,null,null,null,null,null,0,0,0,0.0
1735701720000,93809.3,93831.6,93800.0,93800.0,94439.5,93485.8,93812.7,93808.9,93816.4,0,0,0,-1,-1,null,null,null,null,null,0,0,0,0.0
1735701780000,93800.0,93822.7,93800.0,93812.7,94439.5,93485.8,93808.9,93816.4,93821.2,0,0,0,-1,-1,null,null,null,null,null,0,0,0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1740009120000,96564.3,96564.4,96531.5,96531.6,96786.4,96021.5,96522.4,96525.4,96525.4,0,0,0,0,0,null,null,null,null,null,0,0,0,0.0
1740009180000,96531.5,96531.6,96513.6,96522.4,96786.4,96021.5,96525.4,96525.4,96533.9,0,0,0,0,0,null,null,null,null,null,0,0,0,0.0
1740009240000,96522.5,96525.5,96513.6,96525.4,96786.4,96021.5,96525.4,96533.9,96604.5,0,0,0,0,0,null,null,null,null,null,0,0,0,0.0


# Determining Approach 3

In [7]:
import polars as pl

def track_sequence(df):

    # Initialize columns
    df = df.with_columns([
        pl.lit(None, dtype=pl.Utf8).alias("sequence"),
        pl.lit(0.0, dtype=pl.Float64).alias("upper_long_zone"),
        pl.lit(0.0, dtype=pl.Float64).alias("lower_long_zone"),
        pl.lit(0.0, dtype=pl.Float64).alias("lower_short_zone"),
        pl.lit(0.0, dtype=pl.Float64).alias("upper_short_zone"),
        pl.lit(0, dtype=pl.Int8).alias("long_zone"),
        pl.lit(0, dtype=pl.Int8).alias("short_zone"),
        pl.lit(0, dtype=pl.Int8).alias("target"),
        pl.lit(0, dtype=pl.Float64).alias("target_price"),
        pl.lit(0, dtype=pl.Int8).alias("first_sequence"),
        pl.lit(0, dtype=pl.Float64).alias("max_close"),
        pl.lit(0, dtype=pl.Float64).alias("min_close"),
    ])

    sequences = []
    upper_long_zones = []
    lower_long_zones = []
    lower_short_zones = []
    upper_short_zones = []
    long_zones = []
    short_zones = []
    targets = []
    target_prices = []
    first_sequences = []
    max_closes = []
    min_closes = []

    prev_seq = None
    upper_long_zone = 0.0
    lower_long_zone = 0.0
    lower_short_zone = 0.0
    upper_short_zone = 0.0
    target_1 = 0.0
    target_2 = 0.0
    target_3 = 0.0
    target_4 = 0.0
    target = 0
    target_price = 0.0
    long_zone = 0
    short_zone = 0
    row_count = 0
    prev_min_close = float('inf')  # Initialize to a high value
    prev_max_close = -float('inf') # Initialize to a low value
    

    for i, row in enumerate(df.iter_rows(named=True)):

        prev_100_hit = row["prev_100_hit"]
        three_candles_hit = row["3_candles_hit"]
        fast_reversal_breakout = row["fast_reversal_breakout"]
        prev_100_min_close = row["prev_100_min_close"]
        prev_100_max_close = row["prev_100_max_close"]
        next_1_close = row["next_1_close"]
        next_2_close = row["next_2_close"]
        next_3_close = row["next_3_close"]
        close = row["close"]
        low = row["low"]
        high = row["high"]
        

        first_sequence = 0
        max_close = 0
        min_close = 0
        
        if prev_seq is None:
            if prev_100_hit == -1 and three_candles_hit == -1 and fast_reversal_breakout == 1:
                seq = "Unconfirmed Break Up"
                first_sequence = 1
                long_zone = 0
                short_zone = 0
                target = 0
                target_price = 0.0
                prev_max_close = 0
                # Mark the buy zone
                upper_long_zone = prev_100_min_close
                lower_long_zone = min(next_1_close,next_2_close,next_3_close)
                lower_short_zone = 0.0
                upper_short_zone = 0.0
            elif prev_100_hit == 1 and three_candles_hit == 1 and fast_reversal_breakout == -1:
                seq = "Unconfirmed Break Down"
                first_sequence = 1
                long_zone = 0
                short_zone = 0
                target = 0
                target_price = 0.0
                prev_min_close = 200000
                # Mark the buy zone
                lower_short_zone = prev_100_max_close
                upper_short_zone = max(next_1_close,next_2_close,next_3_close)
                upper_long_zone = 0.0
                lower_long_zone = 0.0
            else:
                seq = None

        elif prev_seq == "Unconfirmed Break Up":
            if prev_100_hit == 1 and three_candles_hit == 1 and prev_100_max_close > upper_long_zone:
                seq = "Confirmed Break Up"
                first_sequence = 1
                row_count = 1
                target_1 = prev_100_max_close
                target_2 = max(next_1_close,next_2_close,next_3_close)
                target_3 = upper_long_zone * (1 + (1.5 * ((prev_100_max_close - upper_long_zone) / upper_long_zone)))
                target_4 = upper_long_zone * (1 + (2 * ((prev_100_max_close - upper_long_zone) / upper_long_zone)))
            elif three_candles_hit == 1 and fast_reversal_breakout == -1:
                seq = "Unconfirmed Break Down"
                first_sequence = 1
                long_zone = 0
                short_zone = 0
                target = 0
                target_price = 0.0
                prev_min_close = 200000
                # Mark the buy zone
                lower_short_zone = prev_100_max_close
                upper_short_zone = max(next_1_close,next_2_close,next_3_close)
                upper_long_zone = 0.0
                lower_long_zone = 0.0
            # elif prev_100_hit == -1 :
            #     # Reset sequence
            #     seq = None
            #     # Reset the buy zone
            #     upper_long_zone = 0.0
            #     lower_long_zone = 0.0
            else:
                seq = prev_seq

        elif prev_seq == "Unconfirmed Break Down":
            if prev_100_hit == -1 and three_candles_hit == -1 and prev_100_min_close < lower_short_zone:
                seq = "Confirmed Break Down"
                first_sequence = 1
                row_count = 1
                target_1 = prev_100_min_close
                target_2 = min(next_1_close,next_2_close,next_3_close)
                target_3 = lower_short_zone * (1 - (1.5 * ((lower_short_zone - prev_100_min_close) / lower_short_zone)))
                target_4 = lower_short_zone * (1 - (2 * ((lower_short_zone - prev_100_min_close) / lower_short_zone)))
            elif three_candles_hit == -1 and fast_reversal_breakout == 1:
                seq = "Unconfirmed Break Up"
                first_sequence = 1
                long_zone = 0
                short_zone = 0
                target = 0
                target_price = 0.0
                prev_max_close = 0
                # Mark the buy zone
                upper_long_zone = prev_100_min_close
                lower_long_zone = min(next_1_close,next_2_close,next_3_close)
                lower_short_zone = 0.0
                upper_short_zone = 0.0
            # elif prev_100_hit == 1 :
            #     # Reset sequence
            #     seq = None
            #     # Reset the buy zone
            #     lower_short_zone = 0.0
            #     upper_short_zone = 0.0
            else:
                seq = prev_seq

        elif prev_seq == "Confirmed Break Up":
            short_zone = 0
            if row_count < 1500 :
                row_count += 1 # add the counter
                # Check if entering long_zone
                if long_zone == 0 and (close <= upper_long_zone):
                    long_zone = 1
                    prev_min_close = close  # Initialize with current close
                    first_sequence = 1
                if long_zone == 1:
                    # Track the minimum close
                    min_close = min(close, prev_min_close)
                    prev_min_close = min_close
                    # Check targets
                    if high >= target_4 and target != 4:
                        target = 4
                        target_price = target_4
                        first_sequence = 1
                    elif high >= target_3 and target < 3:
                        target = 3
                        target_price = target_3
                        first_sequence = 1
                    elif high >= target_2 and target < 2:
                        target = 2
                        target_price = target_2
                        first_sequence = 1
                    elif high >= target_1 and target < 1:
                        target = 1
                        target_price = target_1
                        first_sequence = 1
                    elif row_count == 1499 and target == 0:
                        target = 5
                        target_price = close
                        first_sequence = 1
                elif row_count == 1499 and target == 0:
                    target = 5
                    target_price = close
                    first_sequence = 1
            else:
                seq = None

        elif prev_seq == "Confirmed Break Down":
            long_zone = 0
            if row_count < 1500:
                row_count += 1
                # Check if entering short_zone
                if short_zone == 0 and (close >= lower_short_zone):
                    short_zone = 1
                    prev_max_close = close  # Initialize with current close
                    first_sequence = 1
                if short_zone == 1:
                    # Track the maximum close
                    max_close = max(close, prev_max_close)
                    prev_max_close = max_close
                    # Check targets
                    if low <= target_4 and target != 4:
                        target = 4
                        target_price = target_4
                        first_sequence = 1
                    elif low <= target_3 and target < 3:
                        target = 3
                        target_price = target_3
                        first_sequence = 1
                    elif low <= target_2 and target < 2:
                        target = 2
                        target_price = target_2
                        first_sequence = 1
                    elif low <= target_1 and target < 1:
                        target = 1
                        target_price = target_1
                        first_sequence = 1
                    elif row_count == 1499 and target == 0:
                        target = 5
                        target_price = close
                        first_sequence = 1
                elif row_count == 1499 and target == 0:
                    target = 5
                    target_price = close
                    first_sequence = 1
            else:
                seq = None
            
        sequences.append(seq)
        first_sequences.append(first_sequence)
        targets.append(target)
        upper_long_zones.append(upper_long_zone)
        lower_long_zones.append(lower_long_zone)
        lower_short_zones.append(lower_short_zone)
        upper_short_zones.append(upper_short_zone)
        long_zones.append(long_zone)
        short_zones.append(short_zone)
        target_prices.append(target_price)
        max_closes.append(max_close)
        min_closes.append(min_close)

        prev_seq = seq  # Carry forward the sequence to the next row

    return df.with_columns([
        pl.Series("sequence", sequences, dtype=pl.Utf8),
        pl.Series("upper_long_zone", upper_long_zones, dtype=pl.Float64),
        pl.Series("lower_long_zone", lower_long_zones, dtype=pl.Float64),
        pl.Series("upper_short_zone", upper_short_zones, dtype=pl.Float64),
        pl.Series("lower_short_zone", lower_short_zones, dtype=pl.Float64),
        pl.Series("long_zone", long_zones, dtype=pl.Int8),
        pl.Series("short_zone", short_zones, dtype=pl.Int8),
        pl.Series("target", targets, dtype=pl.Int8),
        pl.Series("target_price", target_prices, dtype=pl.Float64),
        pl.Series("first_sequence", first_sequences, dtype=pl.Int8),
        pl.Series("max_close", max_closes, dtype=pl.Float64),
        pl.Series("min_close", min_closes, dtype=pl.Float64)
    ])

# Apply function
dfs_sequenced = track_sequence(dfs_flagged)
dfs_sequenced

opentime,open,high,low,close,prev_100_max_close,prev_100_min_close,next_1_close,next_2_close,next_3_close,prev_100_hit,3_candles_hit,next_20_hit_up,next_20_hit_down,fast_reversal_breakout,sequence,upper_long_zone,lower_long_zone,lower_short_zone,upper_short_zone,long_zone,short_zone,target,target_price,first_sequence,max_close,min_close
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32,i32,i32,i32,i32,str,f64,f64,f64,f64,i8,i8,i8,f64,i8,f64,f64
1704070140000,0.08996,0.08996,0.08993,0.08995,0.09007,0.08961,0.08995,0.08989,0.08987,0,0,1,-1,1,null,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0.0
1704070200000,0.08995,0.08997,0.08995,0.08995,0.09007,0.08967,0.08989,0.08987,0.0899,0,0,1,-1,1,null,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0.0
1704070260000,0.08995,0.08995,0.08989,0.08989,0.09007,0.08967,0.08987,0.0899,0.08991,0,0,1,-1,1,null,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0.0
1704070320000,0.08988,0.08991,0.08987,0.08987,0.09007,0.08978,0.0899,0.08991,0.08992,0,0,1,-1,1,null,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0.0
1704070380000,0.08988,0.08991,0.08987,0.0899,0.09007,0.08978,0.08991,0.08992,0.08996,0,0,1,-1,1,null,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1735689180000,0.31652,0.31656,0.31626,0.31626,0.31682,0.31524,0.3163,0.31642,0.31622,0,0,0,0,0,"""Confirmed Break Up""",0.31713,0.31642,0.0,0.0,1,0,4,0.31973,0,0.0,0.31524
1735689240000,0.31626,0.31639,0.31626,0.3163,0.31682,0.31524,0.31642,0.31622,0.3159,0,0,0,0,0,"""Confirmed Break Up""",0.31713,0.31642,0.0,0.0,1,0,4,0.31973,0,0.0,0.31524
1735689300000,0.3163,0.31643,0.31617,0.31642,0.31682,0.31524,0.31622,0.3159,0.31597,0,0,0,0,0,"""Confirmed Break Up""",0.31713,0.31642,0.0,0.0,1,0,4,0.31973,0,0.0,0.31524


# Approach 3 with TP and SL

In [171]:
def summarize(df):   
    main_sequences = df.filter(
        (pl.col("first_sequence") == 1)
    )

    # Step 1: Assign sequence_set_id
    set_id = 1
    sequence_set_ids = []
    for row in main_sequences.iter_rows(named=True):
        if "Unconfirmed" in row["sequence"]:
            set_id += 1
        sequence_set_ids.append(set_id)

    main_sequences = main_sequences.with_columns(pl.Series("sequence_set_id", sequence_set_ids))

    # Step 2: Filter for sequences containing "Confirmed"
    filtered_main_sequences = main_sequences.filter(pl.col("sequence").str.contains("Confirmed"))

    # Step 3: Keep only the row with the highest target per sequence_set_id
    filtered_main_sequences = (
        filtered_main_sequences.sort("target", descending=True)
        .unique(subset=["sequence_set_id"], keep="first")
    )

    # Step 4: Select only the specified columns
    final_result = filtered_main_sequences.select([
        "close", "sequence", "upper_long_zone", "lower_long_zone",
        "lower_short_zone", "upper_short_zone","target_price", "target", "sequence_set_id",
        "max_close", "min_close"
    ])

    # Display result
    final_sum = final_result.with_columns(
        pl.when(pl.col("sequence") == "Confirmed Break Up")
        .then((pl.col("target_price") - pl.col("upper_long_zone")) / pl.col("upper_long_zone") * 100)
        .when(pl.col("sequence") == "Confirmed Break Down")
        .then((pl.col("lower_short_zone") - pl.col("target_price")) / pl.col("lower_short_zone") * 100)
        .otherwise(0)
        .alias("pnl")
    )

    final_sum = final_sum.with_columns(
        pl.when((pl.col("target") == 0) & (pl.col("sequence") == "Confirmed Break Up"))
        .then((pl.col("close") - pl.col("upper_long_zone")) / pl.col("upper_long_zone") * 100)
        .when((pl.col("target") == 0) & (pl.col("sequence") == "Confirmed Break Down"))
        .then((pl.col("lower_short_zone") - pl.col("close")) / pl.col("lower_short_zone") * 100)
        .otherwise(pl.col("pnl"))  # Keep existing values unchanged
        .alias("pnl")
    )

    final_sum = final_sum.with_columns(
        (pl.col("pnl").cast(pl.Float64) - 0.04).alias("nett_pnl")
    )

    final_sum = final_sum.with_columns(
        pl.when((pl.col("nett_pnl") >= 0))
        .then(1)
        .otherwise(0)
        .alias("win")
    )

    final_sum = final_sum.with_columns(
    pl.when((pl.col("sequence") == "Confirmed Break Up") & (pl.col("min_close") != 0) & (pl.col("upper_long_zone") != 0))
    .then((pl.col("min_close") - pl.col("upper_long_zone")) / pl.col("upper_long_zone") * 100)
    .when((pl.col("sequence") == "Confirmed Break Down") & (pl.col("max_close") != 0) & (pl.col("lower_short_zone") != 0))
    .then((pl.col("lower_short_zone") - pl.col("max_close")) / pl.col("lower_short_zone") * 100)
    .otherwise(0)
    .alias("exceed_buy_zone_by")
    )
    
    return final_sum

# Apply function
dfs_sequenced = track_sequence(dfs_flagged)
dfs_sequenced

year,month,day,hour,minute,open,high,low,close,prev_100_max_close,prev_100_min_close,next_1_close,next_2_close,next_3_close,prev_100_hit,3_candles_hit,next_20_hit_up,next_20_hit_down,fast_reversal_breakout,sequence,upper_long_zone,lower_long_zone,lower_short_zone,upper_short_zone,long_zone,short_zone,target,target_price,first_sequence,max_close,min_close,tp_sl
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32,i32,i32,i32,i32,str,f64,f64,f64,f64,i8,i8,i8,f64,i8,f64,f64,i8
2024.0,1.0,1.0,0.0,49.0,42472.6,42472.6,42464.8,42464.8,42592.1,42346.8,42485.4,42483.3,42488.3,0,0,1,0,1,null,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0.0,null
2024.0,1.0,1.0,0.0,50.0,42464.8,42485.8,42461.1,42485.4,42592.1,42363.5,42483.3,42488.3,42488.3,0,0,1,0,1,null,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0.0,null
2024.0,1.0,1.0,0.0,51.0,42485.3,42488.4,42477.9,42483.3,42592.1,42369.7,42488.3,42488.3,42507.6,0,0,1,0,1,null,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0.0,null
2024.0,1.0,1.0,0.0,52.0,42483.4,42488.4,42483.3,42488.3,42592.1,42409.4,42488.3,42507.6,42514.8,0,0,1,0,1,null,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0.0,null
2024.0,1.0,1.0,0.0,53.0,42488.3,42488.4,42485.8,42488.3,42592.1,42409.4,42507.6,42514.8,42524.6,0,0,1,0,1,null,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0.0,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024.0,12.0,31.0,23.0,52.0,93624.8,93649.0,93589.7,93649.0,93692.1,93376.0,93622.4,93604.0,93635.1,0,0,0,0,0,"""Confirmed Break Down""",0.0,0.0,95817.4,95947.9,0,0,0,0.0,0,0.0,0.0,null
2024.0,12.0,31.0,23.0,53.0,93649.0,93680.2,93622.4,93622.4,93692.1,93376.0,93604.0,93635.1,93592.6,0,0,0,0,0,"""Confirmed Break Down""",0.0,0.0,95817.4,95947.9,0,0,0,0.0,0,0.0,0.0,null
2024.0,12.0,31.0,23.0,54.0,93622.5,93638.1,93604.0,93604.0,93692.1,93399.1,93635.1,93592.6,93569.0,0,0,0,0,0,"""Confirmed Break Down""",0.0,0.0,95817.4,95947.9,0,0,0,0.0,0,0.0,0.0,null


# Validate Sequence

In [8]:
import polars as pl

def validate_sequences(df):
    errors = []
    
    prev_seq = None  # Track the previous row's sequence
    for idx, row in enumerate(df.iter_rows(named=True)):
        seq = row["sequence"]

        if seq == "Confirmed Break Up":
            if prev_seq not in ["Confirmed Break Up", "Unconfirmed Break Up"]:
                errors.append(f"Row {idx}: Confirmed Break Up follows {prev_seq}")

        elif seq == "Confirmed Break Down":
            if prev_seq not in ["Confirmed Break Down", "Unconfirmed Break Down"]:
                errors.append(f"Row {idx}: Confirmed Break Down follows {prev_seq}")

        prev_seq = seq  # Update previous sequence

    return errors

# Run the validation
errors = validate_sequences(dfs_sequenced)

# Output results
if errors:
    print("Validation Errors Found:")
    for err in errors:
        print(err)
else:
    print("Validation Passed: No errors found!")

Validation Passed: No errors found!


# Summarize

In [208]:
main_sequences = dfs_sequenced.filter(
    (pl.col("first_sequence") == 1)
)

# Step 1: Assign sequence_set_id
set_id = 1
sequence_set_ids = []
for row in main_sequences.iter_rows(named=True):
    if "Unconfirmed" in row["sequence"]:
        set_id += 1
    sequence_set_ids.append(set_id)

main_sequences = main_sequences.with_columns(pl.Series("sequence_set_id", sequence_set_ids))

# Step 2: Filter for sequences containing "Confirmed"
filtered_main_sequences = main_sequences.filter(pl.col("sequence").str.contains("Confirmed"))

# Step 3: Keep only the row with the highest target per sequence_set_id
filtered_main_sequences = (
    filtered_main_sequences.sort("target", descending=True)
    .unique(subset=["sequence_set_id"], keep="first")
)

# Step 4: Select only the specified columns
final_result = filtered_main_sequences.select([
    "close", "sequence", "upper_long_zone", "lower_long_zone",
    "lower_short_zone", "upper_short_zone","target_price", "target", "sequence_set_id",
    "max_close", "min_close","tp_sl"
])

# Display result
final_sum = final_result.with_columns(
    pl.when(pl.col("sequence") == "Confirmed Break Up")
    .then((pl.col("target_price") - pl.col("upper_long_zone")) / pl.col("upper_long_zone") * 100)
    .when(pl.col("sequence") == "Confirmed Break Down")
    .then((pl.col("lower_short_zone") - pl.col("target_price")) / pl.col("lower_short_zone") * 100)
    .otherwise(0)
    .alias("pnl")
)

final_sum = final_sum.with_columns(
    pl.when((pl.col("target") == 0) & (pl.col("sequence") == "Confirmed Break Up"))
    .then((pl.col("close") - pl.col("upper_long_zone")) / pl.col("upper_long_zone") * 100)
    .when((pl.col("target") == 0) & (pl.col("sequence") == "Confirmed Break Down"))
    .then((pl.col("lower_short_zone") - pl.col("close")) / pl.col("lower_short_zone") * 100)
    .otherwise(pl.col("pnl"))  # Keep existing values unchanged
    .alias("pnl")
)

final_sum = final_sum.with_columns(
    (pl.col("pnl").cast(pl.Float64) - 0.04).alias("nett_pnl")
)

final_sum = final_sum.with_columns(
    pl.when((pl.col("nett_pnl") >= 0))
    .then(1)
    .otherwise(0)
    .alias("win")
)

final_sum = final_sum.with_columns(
pl.when((pl.col("sequence") == "Confirmed Break Up") & (pl.col("min_close") != 0) & (pl.col("upper_long_zone") != 0))
.then((pl.col("min_close") - pl.col("upper_long_zone")) / pl.col("upper_long_zone") * 100)
.when((pl.col("sequence") == "Confirmed Break Down") & (pl.col("max_close") != 0) & (pl.col("lower_short_zone") != 0))
.then((pl.col("lower_short_zone") - pl.col("max_close")) / pl.col("lower_short_zone") * 100)
.otherwise(0)
.alias("exceed_buy_zone_by")
)

final_sum = final_sum.with_columns(
pl.when(pl.col("tp_sl") == -1)
.then(-3)
.otherwise(pl.col("exceed_buy_zone_by"))
.alias("exceed_buy_zone_by")
)

final_sum

close,sequence,upper_long_zone,lower_long_zone,lower_short_zone,upper_short_zone,target_price,target,sequence_set_id,max_close,min_close,tp_sl,pnl,nett_pnl,win,exceed_buy_zone_by
f64,str,f64,f64,f64,f64,f64,i8,i64,f64,f64,i8,f64,f64,i32,f64
65925.2,"""Confirmed Break Up""",65955.5,65920.9,0.0,0.0,65964.3,4,137,0.0,65925.2,null,0.013342,-0.026658,0,-0.04594
44129.4,"""Confirmed Break Up""",45381.5,45310.1,0.0,0.0,44020.055,5,3,0.0,44129.4,-1,-3.0,-3.04,0,-3.0
44145.0,"""Confirmed Break Up""",43524.1,43444.5,0.0,0.0,44145.0,5,6,0.0,0.0,null,1.426566,1.386566,1,0.0
63110.1,"""Confirmed Break Up""",62984.9,62974.1,0.0,0.0,63114.7,4,268,0.0,62935.6,null,0.206081,0.166081,1,-0.078273
62053.3,"""Confirmed Break Up""",60887.4,60814.0,0.0,0.0,62053.3,5,134,0.0,0.0,null,1.914846,1.874846,1,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
56037.7,"""Confirmed Break Up""",57751.8,57720.0,0.0,0.0,56019.246,5,253,0.0,56037.7,-1,-3.0,-3.04,0,-3.0
54482.4,"""Confirmed Break Up""",54294.0,54263.7,0.0,0.0,54505.8,4,256,0.0,53635.1,null,0.390098,0.350098,1,-1.213578
56486.3,"""Confirmed Break Up""",56241.5,56130.7,0.0,0.0,56472.7,4,259,0.0,55560.8,null,0.411084,0.371084,1,-1.210316


In [213]:
# Compute required metrics
nett_pnl_sum = final_sum["nett_pnl"].sum()
sequence_set_id_count = final_sum["sequence_set_id"].len()
win_sum = final_sum["win"].sum()

final_sum_win = final_sum.filter((pl.col("win") == 1))
max_buy_zone_exceed_win = final_sum_win["exceed_buy_zone_by"].min()
mean_buy_zone_exceed_win = final_sum_win["exceed_buy_zone_by"].mean()
median_buy_zone_exceed_win = final_sum_win["exceed_buy_zone_by"].median()

final_sum_lose = final_sum.filter((pl.col("win") != 1))
max_buy_zone_exceed_lose = final_sum_lose["exceed_buy_zone_by"].min()
mean_buy_zone_exceed_lose = final_sum_lose["exceed_buy_zone_by"].mean()
median_buy_zone_exceed_lose = final_sum_lose["exceed_buy_zone_by"].median()

# Count TP and SL
tp_count = final_sum.filter(pl.col("tp_sl") == 1).height
sl_count = final_sum.filter(pl.col("tp_sl") == -1).height

# Compute win rate
win_rate = (win_sum / sequence_set_id_count) * 100

# Get target value counts and percentages
target_percentages = (
    final_sum["target"]
    .value_counts()
    .with_columns((pl.col("count") / sequence_set_id_count * 100).alias("percentage"))
)

# Convert to dictionary, ensuring correct unpacking from tuples
target_pct_dict = {target: percentage for target, count, percentage in target_percentages.rows()}

# Extract values with a default of 0 if missing
target_4_pct = target_pct_dict.get(4, 0)
target_3_pct = target_pct_dict.get(3, 0) + target_4_pct
target_2_pct = target_pct_dict.get(2, 0) + target_3_pct
target_1_pct = target_pct_dict.get(1, 0) + target_2_pct

# Print results
print(f"Sum of nett_pnl: {nett_pnl_sum:.4f} %")
print(f"Total Positions: {sequence_set_id_count}")
print(f"Win rate: {win_rate:.2f} %")
print(f"TP/SL: {tp_count} / {sl_count} ")
print(f"Target 1: {target_1_pct:.2f}% | Target 2: {target_2_pct:.2f}% | Target 3: {target_3_pct:.2f}% | Target 4: {target_4_pct:.2f}%")
print(f"Exceed Buy Zone Win - Min {max_buy_zone_exceed_win:.2f}% | Mean: {mean_buy_zone_exceed_win:.2f}% | Median: {median_buy_zone_exceed_win:.2f}%")
print(f"Exceed Buy Zone Lose - Min {max_buy_zone_exceed_lose:.2f}% | Mean: {mean_buy_zone_exceed_lose:.2f}% | Median: {median_buy_zone_exceed_lose:.2f}%")

Sum of nett_pnl: 136.5004 %
Total Positions: 321
Win rate: 77.57 %
TP/SL: 0 / 58 
Target 1: 59.19% | Target 2: 57.63% | Target 3: 57.32% | Target 4: 53.58%
Exceed Buy Zone Win - Min -2.34% | Mean: -0.37% | Median: -0.12%
Exceed Buy Zone Lose - Min -3.00% | Mean: -2.60% | Median: -3.00%


# All Functions

In [9]:
def features(df,prev_row=100):
    import polars as pl

    # Previous 100 rows max close & min close
    dfs_featured = df.with_columns(
        pl.col("close").rolling_max(window_size=prev_row).alias("prev_100_max_close"),
        pl.col("close").rolling_min(window_size=prev_row).alias("prev_100_min_close")
    )

    # Next row close (-1, -2, -3)
    dfs_featured = dfs_featured.with_columns(
        pl.col("close").shift(-1).alias("next_1_close"),
        pl.col("close").shift(-2).alias("next_2_close"),
        pl.col("close").shift(-3).alias("next_3_close")
    )

    dfs_featured = dfs_featured.drop_nulls()

    # Display result
    return dfs_featured

In [10]:
def flags(df,fast_reversal_row = 100):
    # First Break Out
    dfs_flagged = df.with_columns(
        pl.when(pl.col("close") >= pl.col("prev_100_max_close"))
        .then(1)
        .when(pl.col("close") <= pl.col("prev_100_min_close"))
        .then(-1)
        .otherwise(0)
        .alias("prev_100_hit")
    )

    dfs_flagged = dfs_flagged.with_columns(
        pl.when(
            (pl.col("next_1_close") >= pl.col("prev_100_max_close")) &
            (pl.col("next_2_close") >= pl.col("prev_100_max_close")) &
            (pl.col("next_3_close") >= pl.col("prev_100_max_close"))
        )
        .then(1)
        .when(
            (pl.col("next_1_close") <= pl.col("prev_100_min_close")) &
            (pl.col("next_2_close") <= pl.col("prev_100_min_close")) &
            (pl.col("next_3_close") <= pl.col("prev_100_min_close"))
        )
        .then(-1)
        .otherwise(0)
        .alias("3_candles_hit")
    )

    # Get the next 20 rows hit
    dfs_flagged = dfs_flagged.with_columns(
        pl.col("prev_100_hit").rolling_max(window_size=fast_reversal_row).shift(-1*(fast_reversal_row+1)).alias("next_20_hit_up"),
        pl.col("prev_100_hit").rolling_min(window_size=fast_reversal_row).shift(-1*(fast_reversal_row+1)).alias("next_20_hit_down")
    )

    dfs_flagged = dfs_flagged.with_columns(
        pl.col("next_20_hit_up").fill_null(0),
        pl.col("next_20_hit_down").fill_null(0)
    )

    # Fast Reversal Breakout
    dfs_flagged = dfs_flagged.with_columns(
        pl.when(
            (pl.col("next_20_hit_up") == 1)
        )
        .then(1)
        .when(
            (pl.col("next_20_hit_down") == -1)
        )
        .then(-1)
        .otherwise(0)
        .alias("fast_reversal_breakout")
    )

    # Display result
    return dfs_flagged

In [11]:
import polars as pl

def track_sequence(df, n_threshold=500):

    # Initialize columns
    df = df.with_columns([
        pl.lit(None, dtype=pl.Utf8).alias("sequence"),
        pl.lit(0.0, dtype=pl.Float64).alias("upper_long_zone"),
        pl.lit(0.0, dtype=pl.Float64).alias("lower_long_zone"),
        pl.lit(0.0, dtype=pl.Float64).alias("lower_short_zone"),
        pl.lit(0.0, dtype=pl.Float64).alias("upper_short_zone"),
        pl.lit(0, dtype=pl.Int8).alias("long_zone"),
        pl.lit(0, dtype=pl.Int8).alias("short_zone"),
        pl.lit(0, dtype=pl.Int8).alias("target"),
        pl.lit(0, dtype=pl.Float64).alias("target_price"),
        pl.lit(0, dtype=pl.Int8).alias("first_sequence"),
        pl.lit(0, dtype=pl.Float64).alias("max_close"),
        pl.lit(0, dtype=pl.Float64).alias("min_close"),
    ])

    sequences = []
    upper_long_zones = []
    lower_long_zones = []
    lower_short_zones = []
    upper_short_zones = []
    long_zones = []
    short_zones = []
    targets = []
    target_prices = []
    first_sequences = []
    max_closes = []
    min_closes = []

    prev_seq = None
    upper_long_zone = 0.0
    lower_long_zone = 0.0
    lower_short_zone = 0.0
    upper_short_zone = 0.0
    target_1 = 0.0
    target_2 = 0.0
    target_3 = 0.0
    target_4 = 0.0
    target = 0
    target_price = 0.0
    long_zone = 0
    short_zone = 0
    row_count = 0
    prev_min_close = float('inf')  # Initialize to a high value
    prev_max_close = -float('inf') # Initialize to a low value
    

    for i, row in enumerate(df.iter_rows(named=True)):

        prev_100_hit = row["prev_100_hit"]
        three_candles_hit = row["3_candles_hit"]
        fast_reversal_breakout = row["fast_reversal_breakout"]
        prev_100_min_close = row["prev_100_min_close"]
        prev_100_max_close = row["prev_100_max_close"]
        next_1_close = row["next_1_close"]
        next_2_close = row["next_2_close"]
        next_3_close = row["next_3_close"]
        close = row["close"]
        low = row["low"]
        high = row["high"]
        

        first_sequence = 0
        max_close = 0
        min_close = 0
        
        if prev_seq is None:
            if prev_100_hit == -1 and three_candles_hit == -1 and fast_reversal_breakout == 1:
                seq = "Unconfirmed Break Up"
                first_sequence = 1
                long_zone = 0
                short_zone = 0
                target = 0
                target_price = 0.0
                prev_max_close = 0
                # Mark the buy zone
                upper_long_zone = prev_100_min_close
                lower_long_zone = min(next_1_close,next_2_close,next_3_close)
                lower_short_zone = 0.0
                upper_short_zone = 0.0
            elif prev_100_hit == 1 and three_candles_hit == 1 and fast_reversal_breakout == -1:
                seq = "Unconfirmed Break Down"
                first_sequence = 1
                long_zone = 0
                short_zone = 0
                target = 0
                target_price = 0.0
                prev_min_close = 200000
                # Mark the buy zone
                lower_short_zone = prev_100_max_close
                upper_short_zone = max(next_1_close,next_2_close,next_3_close)
                upper_long_zone = 0.0
                lower_long_zone = 0.0
            else:
                seq = None

        elif prev_seq == "Unconfirmed Break Up":
            if prev_100_hit == 1 and three_candles_hit == 1 and prev_100_max_close > upper_long_zone:
                seq = "Confirmed Break Up"
                first_sequence = 1
                row_count = 1
                target_1 = prev_100_max_close
                target_2 = max(next_1_close,next_2_close,next_3_close)
                target_3 = upper_long_zone * (1 + (1.5 * ((prev_100_max_close - upper_long_zone) / upper_long_zone)))
                target_4 = upper_long_zone * (1 + (2 * ((prev_100_max_close - upper_long_zone) / upper_long_zone)))
            elif three_candles_hit == 1 and fast_reversal_breakout == -1:
                seq = "Unconfirmed Break Down"
                first_sequence = 1
                long_zone = 0
                short_zone = 0
                target = 0
                target_price = 0.0
                prev_min_close = 200000
                # Mark the buy zone
                lower_short_zone = prev_100_max_close
                upper_short_zone = max(next_1_close,next_2_close,next_3_close)
                upper_long_zone = 0.0
                lower_long_zone = 0.0
            # elif prev_100_hit == -1 :
            #     # Reset sequence
            #     seq = None
            #     # Reset the buy zone
            #     upper_long_zone = 0.0
            #     lower_long_zone = 0.0
            else:
                seq = prev_seq

        elif prev_seq == "Unconfirmed Break Down":
            if prev_100_hit == -1 and three_candles_hit == -1 and prev_100_min_close < lower_short_zone:
                seq = "Confirmed Break Down"
                first_sequence = 1
                row_count = 1
                target_1 = prev_100_min_close
                target_2 = min(next_1_close,next_2_close,next_3_close)
                target_3 = lower_short_zone * (1 - (1.5 * ((lower_short_zone - prev_100_min_close) / lower_short_zone)))
                target_4 = lower_short_zone * (1 - (2 * ((lower_short_zone - prev_100_min_close) / lower_short_zone)))
            elif three_candles_hit == -1 and fast_reversal_breakout == 1:
                seq = "Unconfirmed Break Up"
                first_sequence = 1
                long_zone = 0
                short_zone = 0
                target = 0
                target_price = 0.0
                prev_max_close = 0
                # Mark the buy zone
                upper_long_zone = prev_100_min_close
                lower_long_zone = min(next_1_close,next_2_close,next_3_close)
                lower_short_zone = 0.0
                upper_short_zone = 0.0
            # elif prev_100_hit == 1 :
            #     # Reset sequence
            #     seq = None
            #     # Reset the buy zone
            #     lower_short_zone = 0.0
            #     upper_short_zone = 0.0
            else:
                seq = prev_seq

        elif prev_seq == "Confirmed Break Up":
            short_zone = 0
            if row_count < n_threshold :
                row_count += 1 # add the counter
                # Check if entering long_zone
                if long_zone == 0 and (close <= upper_long_zone):
                    long_zone = 1
                    prev_min_close = close  # Initialize with current close
                    first_sequence = 1
                if long_zone == 1:
                    # Track the minimum close
                    min_close = min(close, prev_min_close)
                    prev_min_close = min_close
                    # Check targets
                    if high >= target_4 and target != 4:
                        target = 4
                        target_price = target_4
                        first_sequence = 1
                    elif high >= target_3 and target < 3:
                        target = 3
                        target_price = target_3
                        first_sequence = 1
                    elif high >= target_2 and target < 2:
                        target = 2
                        target_price = target_2
                        first_sequence = 1
                    elif high >= target_1 and target < 1:
                        target = 1
                        target_price = target_1
                        first_sequence = 1
                    elif row_count == (n_threshold - 1) and target == 0:
                        target = 5
                        target_price = close
                        first_sequence = 1
                elif row_count == (n_threshold - 1) and target == 0:
                    target = 5
                    target_price = close
                    first_sequence = 1
            else:
                seq = None

        elif prev_seq == "Confirmed Break Down":
            long_zone = 0
            if row_count < n_threshold:
                row_count += 1
                # Check if entering short_zone
                if short_zone == 0 and (close >= lower_short_zone):
                    short_zone = 1
                    prev_max_close = close  # Initialize with current close
                    first_sequence = 1
                if short_zone == 1:
                    # Track the maximum close
                    max_close = max(close, prev_max_close)
                    prev_max_close = max_close
                    # Check targets
                    if low <= target_4 and target != 4:
                        target = 4
                        target_price = target_4
                        first_sequence = 1
                    elif low <= target_3 and target < 3:
                        target = 3
                        target_price = target_3
                        first_sequence = 1
                    elif low <= target_2 and target < 2:
                        target = 2
                        target_price = target_2
                        first_sequence = 1
                    elif low <= target_1 and target < 1:
                        target = 1
                        target_price = target_1
                        first_sequence = 1
                    elif row_count == (n_threshold - 1) and target == 0:
                        target = 5
                        target_price = close
                        first_sequence = 1
                elif row_count == (n_threshold - 1) and target == 0:
                    target = 5
                    target_price = close
                    first_sequence = 1
            else:
                seq = None
            
        sequences.append(seq)
        first_sequences.append(first_sequence)
        targets.append(target)
        upper_long_zones.append(upper_long_zone)
        lower_long_zones.append(lower_long_zone)
        lower_short_zones.append(lower_short_zone)
        upper_short_zones.append(upper_short_zone)
        long_zones.append(long_zone)
        short_zones.append(short_zone)
        target_prices.append(target_price)
        max_closes.append(max_close)
        min_closes.append(min_close)

        prev_seq = seq  # Carry forward the sequence to the next row

    return df.with_columns([
        pl.Series("sequence", sequences, dtype=pl.Utf8),
        pl.Series("upper_long_zone", upper_long_zones, dtype=pl.Float64),
        pl.Series("lower_long_zone", lower_long_zones, dtype=pl.Float64),
        pl.Series("upper_short_zone", upper_short_zones, dtype=pl.Float64),
        pl.Series("lower_short_zone", lower_short_zones, dtype=pl.Float64),
        pl.Series("long_zone", long_zones, dtype=pl.Int8),
        pl.Series("short_zone", short_zones, dtype=pl.Int8),
        pl.Series("target", targets, dtype=pl.Int8),
        pl.Series("target_price", target_prices, dtype=pl.Float64),
        pl.Series("first_sequence", first_sequences, dtype=pl.Int8),
        pl.Series("max_close", max_closes, dtype=pl.Float64),
        pl.Series("min_close", min_closes, dtype=pl.Float64)
    ])

In [12]:
# With TP and SL
import polars as pl

def track_sequence(df, n_threshold=500, tp=3, sl=3):
    # Initialize columns
    df = df.with_columns([
        pl.lit(None, dtype=pl.Utf8).alias("sequence"),
        pl.lit(0.0, dtype=pl.Float64).alias("upper_long_zone"),
        pl.lit(0.0, dtype=pl.Float64).alias("lower_long_zone"),
        pl.lit(0.0, dtype=pl.Float64).alias("lower_short_zone"),
        pl.lit(0.0, dtype=pl.Float64).alias("upper_short_zone"),
        pl.lit(0, dtype=pl.Int8).alias("long_zone"),
        pl.lit(0, dtype=pl.Int8).alias("short_zone"),
        pl.lit(0, dtype=pl.Int8).alias("target"),
        pl.lit(0, dtype=pl.Float64).alias("target_price"),
        pl.lit(0, dtype=pl.Int8).alias("first_sequence"),
        pl.lit(0, dtype=pl.Float64).alias("max_close"),
        pl.lit(0, dtype=pl.Float64).alias("min_close"),
        pl.lit(None, dtype=pl.Int8).alias("tp_sl"),  # New column for take-profit/stop-loss
    ])

    # Lists to store column values
    sequences = []
    upper_long_zones = []
    lower_long_zones = []
    lower_short_zones = []
    upper_short_zones = []
    long_zones = []
    short_zones = []
    targets = []
    target_prices = []
    first_sequences = []
    max_closes = []
    min_closes = []
    tp_sls = []  # List to store tp_sl values

    # Initialize tracking variables
    prev_seq = None
    upper_long_zone = 0.0
    lower_long_zone = 0.0
    lower_short_zone = 0.0
    upper_short_zone = 0.0
    target_1 = 0.0
    target_2 = 0.0
    target_3 = 0.0
    target_4 = 0.0
    target = 0
    target_price = 0.0
    long_zone = 0
    short_zone = 0
    row_count = 0
    prev_min_close = float('inf')  # Initialize to a high value
    prev_max_close = -float('inf')  # Initialize to a low value
    tp_sl = None  # Initialize tp_sl
    buy_zone_touched = False  # Flag to track if buy zone is touched

    for i, row in enumerate(df.iter_rows(named=True)):
        # Extract row values
        prev_100_hit = row["prev_100_hit"]
        three_candles_hit = row["3_candles_hit"]
        fast_reversal_breakout = row["fast_reversal_breakout"]
        prev_100_min_close = row["prev_100_min_close"]
        prev_100_max_close = row["prev_100_max_close"]
        next_1_close = row["next_1_close"]
        next_2_close = row["next_2_close"]
        next_3_close = row["next_3_close"]
        close = row["close"]
        low = row["low"]
        high = row["high"]

        first_sequence = 0
        max_close = 0
        min_close = 0

        # Reset tp_sl and buy_zone_touched if sequence is reset
        if prev_seq is None:
            tp_sl = None
            buy_zone_touched = False

        # Sequence logic
        if prev_seq is None:
            if prev_100_hit == -1 and three_candles_hit == -1 and fast_reversal_breakout == 1:
                seq = "Unconfirmed Break Up"
                first_sequence = 1
                long_zone = 0
                short_zone = 0
                target = 0
                target_price = 0.0
                prev_max_close = 0
                # Mark the buy zone
                upper_long_zone = prev_100_min_close
                lower_long_zone = min(next_1_close, next_2_close, next_3_close)
                lower_short_zone = 0.0
                upper_short_zone = 0.0
            elif prev_100_hit == 1 and three_candles_hit == 1 and fast_reversal_breakout == -1:
                seq = "Unconfirmed Break Down"
                first_sequence = 1
                long_zone = 0
                short_zone = 0
                target = 0
                target_price = 0.0
                prev_min_close = 200000
                # Mark the buy zone
                lower_short_zone = prev_100_max_close
                upper_short_zone = max(next_1_close, next_2_close, next_3_close)
                upper_long_zone = 0.0
                lower_long_zone = 0.0
            else:
                seq = None

        elif prev_seq == "Unconfirmed Break Up":
            if prev_100_hit == 1 and three_candles_hit == 1 and prev_100_max_close > upper_long_zone:
                seq = "Confirmed Break Up"
                first_sequence = 1
                row_count = 1
                target_1 = prev_100_max_close
                target_2 = max(next_1_close, next_2_close, next_3_close)
                target_3 = upper_long_zone * (1 + (1.5 * ((prev_100_max_close - upper_long_zone) / upper_long_zone)))
                target_4 = upper_long_zone * (1 + (2 * ((prev_100_max_close - upper_long_zone) / upper_long_zone)))
            elif three_candles_hit == 1 and fast_reversal_breakout == -1:
                seq = "Unconfirmed Break Down"
                first_sequence = 1
                long_zone = 0
                short_zone = 0
                target = 0
                target_price = 0.0
                prev_min_close = 200000
                # Mark the buy zone
                lower_short_zone = prev_100_max_close
                upper_short_zone = max(next_1_close, next_2_close, next_3_close)
                upper_long_zone = 0.0
                lower_long_zone = 0.0
            else:
                seq = prev_seq

        elif prev_seq == "Unconfirmed Break Down":
            if prev_100_hit == -1 and three_candles_hit == -1 and prev_100_min_close < lower_short_zone:
                seq = "Confirmed Break Down"
                first_sequence = 1
                row_count = 1
                target_1 = prev_100_min_close
                target_2 = min(next_1_close, next_2_close, next_3_close)
                target_3 = lower_short_zone * (1 - (1.5 * ((lower_short_zone - prev_100_min_close) / lower_short_zone)))
                target_4 = lower_short_zone * (1 - (2 * ((lower_short_zone - prev_100_min_close) / lower_short_zone)))
            elif three_candles_hit == -1 and fast_reversal_breakout == 1:
                seq = "Unconfirmed Break Up"
                first_sequence = 1
                long_zone = 0
                short_zone = 0
                target = 0
                target_price = 0.0
                prev_max_close = 0
                # Mark the buy zone
                upper_long_zone = prev_100_min_close
                lower_long_zone = min(next_1_close, next_2_close, next_3_close)
                lower_short_zone = 0.0
                upper_short_zone = 0.0
            else:
                seq = prev_seq

        elif prev_seq == "Confirmed Break Up":
            short_zone = 0
            if row_count < n_threshold:
                row_count += 1
                # Check if entering long_zone
                if long_zone == 0 and (close <= upper_long_zone):
                    long_zone = 1
                    prev_min_close = close  # Initialize with current close
                    first_sequence = 1
                    buy_zone_touched = True  # Mark buy zone as touched

                if long_zone == 1 and buy_zone_touched:
                    # Track the minimum close
                    min_close = min(close, prev_min_close)
                    prev_min_close = min_close

                    # Check for TP or SL conditions
                    if tp_sl is None:  # Only check if tp_sl is not already set
                        # Calculate percentage difference
                        high_diff = (high - upper_long_zone) / upper_long_zone * 100
                        low_diff = (upper_long_zone - low) / upper_long_zone * 100

                        if high_diff >= tp:  # TP condition (3% above buy zone)
                            tp_sl = 1  # TP
                            target = 5
                            target_price = upper_long_zone * (1 + tp/100)  # TP price
                            first_sequence = 1
                        elif low_diff >= sl:  # SL condition (3% below buy zone)
                            tp_sl = -1  # SL
                            target = 5
                            target_price = upper_long_zone * (1 - sl/100)  # SL price
                            first_sequence = 1

                    # Check targets
                    if high >= target_4 and target != 4:
                        target = 4
                        target_price = target_4
                        first_sequence = 1
                    elif high >= target_3 and target < 3:
                        target = 3
                        target_price = target_3
                        first_sequence = 1
                    elif high >= target_2 and target < 2:
                        target = 2
                        target_price = target_2
                        first_sequence = 1
                    elif high >= target_1 and target < 1:
                        target = 1
                        target_price = target_1
                        first_sequence = 1
                    elif row_count == (n_threshold - 1) and target == 0:
                        target = 5
                        target_price = close
                        first_sequence = 1
                elif row_count == (n_threshold - 1) and target == 0:
                    target = 5
                    target_price = close
                    first_sequence = 1
            else:
                seq = None

        elif prev_seq == "Confirmed Break Down":
            long_zone = 0
            if row_count < n_threshold:
                row_count += 1
                # Check if entering short_zone
                if short_zone == 0 and (close >= lower_short_zone):
                    short_zone = 1
                    prev_max_close = close  # Initialize with current close
                    first_sequence = 1
                    buy_zone_touched = True  # Mark buy zone as touched

                if short_zone == 1 and buy_zone_touched:
                    # Track the maximum close
                    max_close = max(close, prev_max_close)
                    prev_max_close = max_close

                    # Check for TP or SL conditions
                    if tp_sl is None:  # Only check if tp_sl is not already set
                        # Calculate percentage difference
                        high_diff = (high - lower_short_zone) / lower_short_zone * 100
                        low_diff = (lower_short_zone - low) / lower_short_zone * 100

                        if high_diff >= sl:  # SL condition (3% above buy zone)
                            tp_sl = -1  # SL
                            target = 5
                            target_price = lower_short_zone * (1 + sl/100)  # TP price
                            first_sequence = 1
                        elif low_diff >= tp:  # TP condition (3% below buy zone)
                            tp_sl = 1  # TP
                            target = 5
                            target_price = lower_short_zone * (1 - tp/100)  # SL price
                            first_sequence = 1

                    # Check targets
                    if low <= target_4 and target != 4:
                        target = 4
                        target_price = target_4
                        first_sequence = 1
                    elif low <= target_3 and target < 3:
                        target = 3
                        target_price = target_3
                        first_sequence = 1
                    elif low <= target_2 and target < 2:
                        target = 2
                        target_price = target_2
                        first_sequence = 1
                    elif low <= target_1 and target < 1:
                        target = 1
                        target_price = target_1
                        first_sequence = 1
                    elif row_count == (n_threshold - 1) and target == 0:
                        target = 5
                        target_price = close
                        first_sequence = 1
                elif row_count == (n_threshold - 1) and target == 0:
                    target = 5
                    target_price = close
                    first_sequence = 1
            else:
                seq = None

        # Append values to lists
        sequences.append(seq)
        first_sequences.append(first_sequence)
        targets.append(target)
        upper_long_zones.append(upper_long_zone)
        lower_long_zones.append(lower_long_zone)
        lower_short_zones.append(lower_short_zone)
        upper_short_zones.append(upper_short_zone)
        long_zones.append(long_zone)
        short_zones.append(short_zone)
        target_prices.append(target_price)
        max_closes.append(max_close)
        min_closes.append(min_close)
        tp_sls.append(tp_sl)  # Append tp_sl value

        # Reset sequence if tp_sl is marked
        if tp_sl is not None:
            prev_seq = None
            tp_sl = None
            buy_zone_touched = False
        else:
            prev_seq = seq  # Carry forward the sequence to the next row

    return df.with_columns([
        pl.Series("sequence", sequences, dtype=pl.Utf8),
        pl.Series("upper_long_zone", upper_long_zones, dtype=pl.Float64),
        pl.Series("lower_long_zone", lower_long_zones, dtype=pl.Float64),
        pl.Series("upper_short_zone", upper_short_zones, dtype=pl.Float64),
        pl.Series("lower_short_zone", lower_short_zones, dtype=pl.Float64),
        pl.Series("long_zone", long_zones, dtype=pl.Int8),
        pl.Series("short_zone", short_zones, dtype=pl.Int8),
        pl.Series("target", targets, dtype=pl.Int8),
        pl.Series("target_price", target_prices, dtype=pl.Float64),
        pl.Series("first_sequence", first_sequences, dtype=pl.Int8),
        pl.Series("max_close", max_closes, dtype=pl.Float64),
        pl.Series("min_close", min_closes, dtype=pl.Float64),
        pl.Series("tp_sl", tp_sls, dtype=pl.Int8),  # Add tp_sl column
    ])

In [13]:
def summarize(df):   
    main_sequences = df.filter(
        (pl.col("first_sequence") == 1)
    )

    # Step 1: Assign sequence_set_id
    set_id = 1
    sequence_set_ids = []
    for row in main_sequences.iter_rows(named=True):
        if "Unconfirmed" in row["sequence"]:
            set_id += 1
        sequence_set_ids.append(set_id)

    main_sequences = main_sequences.with_columns(pl.Series("sequence_set_id", sequence_set_ids))

    # Step 2: Filter for sequences containing "Confirmed"
    filtered_main_sequences = main_sequences.filter(pl.col("sequence").str.contains("Confirmed"))

    # Step 3: Keep only the row with the highest target per sequence_set_id
    filtered_main_sequences = (
        filtered_main_sequences.sort("target", descending=True)
        .unique(subset=["sequence_set_id"], keep="first")
    )

    # Step 4: Select only the specified columns
    final_result = filtered_main_sequences.select([
        "close", "sequence", "upper_long_zone", "lower_long_zone",
        "lower_short_zone", "upper_short_zone","target_price", "target", "sequence_set_id",
        "max_close", "min_close","tp_sl"
    ])

    # Display result
    final_sum = final_result.with_columns(
        pl.when(pl.col("sequence") == "Confirmed Break Up")
        .then((pl.col("target_price") - pl.col("upper_long_zone")) / pl.col("upper_long_zone") * 100)
        .when(pl.col("sequence") == "Confirmed Break Down")
        .then((pl.col("lower_short_zone") - pl.col("target_price")) / pl.col("lower_short_zone") * 100)
        .otherwise(0)
        .alias("pnl")
    )

    final_sum = final_sum.with_columns(
        pl.when((pl.col("target") == 0) & (pl.col("sequence") == "Confirmed Break Up"))
        .then((pl.col("close") - pl.col("upper_long_zone")) / pl.col("upper_long_zone") * 100)
        .when((pl.col("target") == 0) & (pl.col("sequence") == "Confirmed Break Down"))
        .then((pl.col("lower_short_zone") - pl.col("close")) / pl.col("lower_short_zone") * 100)
        .otherwise(pl.col("pnl"))  # Keep existing values unchanged
        .alias("pnl")
    )

    final_sum = final_sum.with_columns(
        (pl.col("pnl").cast(pl.Float64) - 0.04).alias("nett_pnl")
    )

    final_sum = final_sum.with_columns(
    pl.when((pl.col("nett_pnl") >= 0))
    .then(1)
    .otherwise(0)
    .alias("win")
)

    final_sum = final_sum.with_columns(
    pl.when((pl.col("sequence") == "Confirmed Break Up") & (pl.col("min_close") != 0) & (pl.col("upper_long_zone") != 0))
    .then((pl.col("min_close") - pl.col("upper_long_zone")) / pl.col("upper_long_zone") * 100)
    .when((pl.col("sequence") == "Confirmed Break Down") & (pl.col("max_close") != 0) & (pl.col("lower_short_zone") != 0))
    .then((pl.col("lower_short_zone") - pl.col("max_close")) / pl.col("lower_short_zone") * 100)
    .otherwise(0)
    .alias("exceed_buy_zone_by")
    )

    final_sum = final_sum.with_columns(
    pl.when(pl.col("tp_sl") == -1)
    .then(-3)
    .otherwise(pl.col("exceed_buy_zone_by"))
    .alias("exceed_buy_zone_by")
    )
    
    return final_sum

In [21]:
def evaluates(final_sum):   
    # Compute required metrics
    nett_pnl_sum = final_sum["nett_pnl"].sum()
    sequence_set_id_count = final_sum["sequence_set_id"].len()
    win_sum = final_sum["win"].sum()

    final_sum_win = final_sum.filter((pl.col("win") == 1))
    max_buy_zone_exceed_win = final_sum_win["exceed_buy_zone_by"].min()
    mean_buy_zone_exceed_win = final_sum_win["exceed_buy_zone_by"].mean()
    median_buy_zone_exceed_win = final_sum_win["exceed_buy_zone_by"].median()

    final_sum_lose = final_sum.filter((pl.col("win") != 1))
    max_buy_zone_exceed_lose = final_sum_lose["exceed_buy_zone_by"].min()
    mean_buy_zone_exceed_lose = final_sum_lose["exceed_buy_zone_by"].mean()
    median_buy_zone_exceed_lose = final_sum_lose["exceed_buy_zone_by"].median()

    # Count TP and SL
    tp_count = final_sum.filter(pl.col("tp_sl") == 1).height
    sl_count = final_sum.filter(pl.col("tp_sl") == -1).height
    sl_count_pct = sl_count / sequence_set_id_count * 100

    # Compute win rate
    win_rate = (win_sum / sequence_set_id_count) * 100

    # Get target value counts and percentages
    target_percentages = (
        final_sum["target"]
        .value_counts()
        .with_columns((pl.col("count") / sequence_set_id_count * 100).alias("percentage"))
    )

    # Convert to dictionary, ensuring correct unpacking from tuples
    target_pct_dict = {target: percentage for target, count, percentage in target_percentages.rows()}

    # Extract values with a default of 0 if missing
    target_4_pct = target_pct_dict.get(4, 0)
    target_3_pct = target_pct_dict.get(3, 0) + target_4_pct
    target_2_pct = target_pct_dict.get(2, 0) + target_3_pct
    target_1_pct = target_pct_dict.get(1, 0) + target_2_pct

    # Print results
    print(f"Sum of nett_pnl: {nett_pnl_sum:.4f}")
    print(f"Win rate: {win_rate:.2f}%")
    print(f"Target 1: {target_1_pct:.2f}% | Target 2: {target_2_pct:.2f}% | Target 3: {target_3_pct:.2f}% | Target 4: {target_4_pct:.2f}%")

    return nett_pnl_sum, win_rate, target_1_pct, target_2_pct, target_3_pct, target_4_pct, sequence_set_id_count, max_buy_zone_exceed_win, mean_buy_zone_exceed_win, median_buy_zone_exceed_win, max_buy_zone_exceed_lose, mean_buy_zone_exceed_lose, median_buy_zone_exceed_lose, tp_count, sl_count, sl_count_pct

# Main Functions

In [32]:
dfs_featured = features(dfs,prev_row=80)
dfs_flagged = flags(dfs_featured,fast_reversal_row=100)
dfs_sequenced = track_sequence(dfs_flagged, 1500)
final_sum = summarize(dfs_sequenced)
nett_pnl_sum, win_rate, target_1_pct, target_2_pct, target_3_pct, target_4_pct, sequence_set_id_count, max_buy_zone_exceed_win, mean_buy_zone_exceed_win, median_buy_zone_exceed_win, max_buy_zone_exceed_lose, mean_buy_zone_exceed_lose, median_buy_zone_exceed_lose, tp_count, sl_count, sl_count_pct = evaluates(final_sum)

Sum of nett_pnl: 346.2852
Win rate: 63.76%
Target 1: 38.42% | Target 2: 37.87% | Target 3: 37.06% | Target 4: 36.24%


# Looping Main

In [23]:
import numpy as np

prev_rows = np.arange(40,310,20)
fast_reversal_rows = np.arange(20,210,20)
n_thresholds = [100, 200, 500, 1000, 1500]

# prev_rows = [40]
# fast_reversal_rows = [100]
# n_thresholds = [1500]

results = []

for prev_row in prev_rows:
    for fast_reversal_row in fast_reversal_rows:
        for n_threshold in n_thresholds:
            dfs_featured = features(dfs, prev_row)
            dfs_flagged = flags(dfs_featured, fast_reversal_row)
            dfs_sequenced = track_sequence(dfs_flagged, n_threshold)
            final_sum = summarize(dfs_sequenced)
            nett_pnl_sum, win_rate, target_1_pct, target_2_pct, target_3_pct, target_4_pct, sequence_set_id_count, max_buy_zone_exceed_win, mean_buy_zone_exceed_win, median_buy_zone_exceed_win, max_buy_zone_exceed_lose, mean_buy_zone_exceed_lose, median_buy_zone_exceed_lose, tp_count, sl_count, sl_count_pct = evaluates(final_sum)

            results.append([prev_row, fast_reversal_row, n_threshold, nett_pnl_sum, win_rate, target_1_pct, target_2_pct, target_3_pct, target_4_pct, sequence_set_id_count,
                            max_buy_zone_exceed_win, mean_buy_zone_exceed_win, median_buy_zone_exceed_win, max_buy_zone_exceed_lose, mean_buy_zone_exceed_lose, median_buy_zone_exceed_lose,
                            tp_count, sl_count, sl_count_pct])

# Convert to Polars DataFrame
df_results = pl.DataFrame(
    results, 
    orient="row",
    schema=["prev_row", "fast_reversal_row", "target_threshold", "nett_pnl_sum", "win_rate", "target_1_pct", "target_2_pct", "target_3_pct", "target_4_pct", "total_trade",
            "max_buy_zone_exceed_win", "mean_buy_zone_exceed_win", "median_buy_zone_exceed_win", "max_buy_zone_exceed_lose", "mean_buy_zone_exceed_lose", "median_buy_zone_exceed_lose",
            "tp_count", "sl_count", "sl_count_pct"]
)

df_results

Sum of nett_pnl: 419.5011
Win rate: 83.16%
Target 1: 20.69% | Target 2: 18.05% | Target 3: 16.23% | Target 4: 12.78%
Sum of nett_pnl: 335.9744
Win rate: 75.39%
Target 1: 30.16% | Target 2: 27.05% | Target 3: 25.50% | Target 4: 21.29%
Sum of nett_pnl: 260.4379
Win rate: 72.09%
Target 1: 40.11% | Target 2: 37.67% | Target 3: 36.04% | Target 4: 31.98%
Sum of nett_pnl: 205.9745
Win rate: 66.45%
Target 1: 45.18% | Target 2: 43.52% | Target 3: 41.53% | Target 4: 40.20%
Sum of nett_pnl: 197.4552
Win rate: 62.13%
Target 1: 45.22% | Target 2: 44.12% | Target 3: 44.12% | Target 4: 42.28%
Sum of nett_pnl: 1018.8290
Win rate: 81.80%
Target 1: 26.91% | Target 2: 23.20% | Target 3: 21.99% | Target 4: 17.74%
Sum of nett_pnl: 820.8883
Win rate: 77.91%
Target 1: 37.97% | Target 2: 33.93% | Target 3: 32.61% | Target 4: 28.76%
Sum of nett_pnl: 517.8738
Win rate: 73.07%
Target 1: 43.92% | Target 2: 41.30% | Target 3: 39.64% | Target 4: 35.50%
Sum of nett_pnl: 359.6965
Win rate: 66.98%
Target 1: 48.58% | T

prev_row,fast_reversal_row,target_threshold,nett_pnl_sum,win_rate,target_1_pct,target_2_pct,target_3_pct,target_4_pct,total_trade,max_buy_zone_exceed_win,mean_buy_zone_exceed_win,median_buy_zone_exceed_win,max_buy_zone_exceed_lose,mean_buy_zone_exceed_lose,median_buy_zone_exceed_lose,tp_count,sl_count,sl_count_pct
i64,i64,i64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,i64,i64,f64
40,20,100,419.501061,83.1643,20.689655,18.052738,16.227181,12.778905,493,-1.691042,-0.085976,0.0,-3.0,-1.017914,-0.77634,2,10,2.028398
40,20,200,335.974449,75.388027,30.155211,27.050998,25.498891,21.286031,451,-2.410321,-0.157505,0.0,-3.0,-1.458495,-1.272169,5,24,5.321508
40,20,500,260.437915,72.086721,40.108401,37.669377,36.04336,31.97832,369,-2.723781,-0.356859,-0.074282,-3.0,-2.393313,-3.0,11,60,16.260163
40,20,1000,205.974531,66.445183,45.182724,43.521595,41.528239,40.199336,301,-2.89071,-0.465702,-0.20373,-3.0,-2.77848,-3.0,11,80,26.578073
40,20,1500,197.455234,62.132353,45.220588,44.117647,44.117647,42.279412,272,-2.802144,-0.625848,-0.307904,-3.0,-2.831549,-3.0,13,91,33.455882
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
300,200,100,436.993537,94.117647,3.208556,2.139037,1.604278,1.604278,187,-1.485661,-0.029989,0.0,-3.0,-1.092621,-1.016364,1,1,0.534759
300,200,200,480.426181,91.891892,5.945946,3.783784,3.243243,2.702703,185,-1.790666,-0.061129,0.0,-3.0,-1.466591,-1.29371,3,4,2.162162
300,200,500,415.076112,82.941176,10.0,8.235294,7.058824,4.705882,170,-1.52408,-0.129231,0.0,-3.0,-1.833794,-1.857003,3,7,4.117647


In [27]:
# Get the row with the max nett_pnl_sum
df_results.filter(pl.col("nett_pnl_sum") == df_results["nett_pnl_sum"].max())

prev_row,fast_reversal_row,target_threshold,nett_pnl_sum,win_rate,target_1_pct,target_2_pct,target_3_pct,target_4_pct,total_trade,max_buy_zone_exceed_win,mean_buy_zone_exceed_win,median_buy_zone_exceed_win,max_buy_zone_exceed_lose,mean_buy_zone_exceed_lose,median_buy_zone_exceed_lose,tp_count,sl_count,sl_count_pct
i64,i64,i64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,i64,i64,f64
40,40,100,1018.828983,81.804397,26.914329,23.199393,21.986353,17.740713,1319,-1.671952,-0.094652,0.0,-3.0,-1.027333,-0.72167,2,24,1.81956


In [28]:
# Get the row with the max total_trade where nett_pnl_sum >= 0
df_results.filter(
    (pl.col("nett_pnl_sum") >= 0) & (pl.col("total_trade") == df_results.filter(pl.col("nett_pnl_sum") >= 0)["total_trade"].max())
)

prev_row,fast_reversal_row,target_threshold,nett_pnl_sum,win_rate,target_1_pct,target_2_pct,target_3_pct,target_4_pct,total_trade,max_buy_zone_exceed_win,mean_buy_zone_exceed_win,median_buy_zone_exceed_win,max_buy_zone_exceed_lose,mean_buy_zone_exceed_lose,median_buy_zone_exceed_lose,tp_count,sl_count,sl_count_pct
i64,i64,i64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,i64,i64,f64
40,40,100,1018.828983,81.804397,26.914329,23.199393,21.986353,17.740713,1319,-1.671952,-0.094652,0.0,-3.0,-1.027333,-0.72167,2,24,1.81956


In [18]:
# Filter profitable rows
profitable_df = df_results.filter(pl.col("win_rate") >= 50)
# Get the row with the maximum opportunities
max_opportunities_profitable_params = profitable_df.sort("total_trade", descending=True).row(0)
# Convert to DataFrame with explicit row orientation
max_opportunities_profitable_df = pl.DataFrame([max_opportunities_profitable_params], schema=df_results.schema, orient="row")
max_opportunities_profitable_df

prev_row,fast_reversal_row,target_threshold,nett_pnl_sum,win_rate,target_1_pct,target_2_pct,target_3_pct,target_4_pct,total_trade,max_buy_zone_exceed_win,mean_buy_zone_exceed_win,median_buy_zone_exceed_win,max_buy_zone_exceed_lose,mean_buy_zone_exceed_lose,median_buy_zone_exceed_lose,tp_count,sl_count,sl_count_pct
i64,i64,i64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,i64,i64,f64
40,60,1500,233.898058,58.370044,42.731278,41.85022,41.409692,40.748899,454,-2.912215,-0.519673,-0.174865,-3.0,-2.8127,-3.0,12,173,38.105727


In [256]:
# Filter profitable rows
profitable_df = df_results.filter(pl.col("target_4_pct") >= 50)
# Get the row with the maximum opportunities
max_opportunities_profitable_params = profitable_df.sort("nett_pnl_sum", descending=True).row(0)
# Convert to DataFrame with explicit row orientation
max_opportunities_profitable_df = pl.DataFrame([max_opportunities_profitable_params], schema=df_results.schema, orient="row")
max_opportunities_profitable_df

prev_row,fast_reversal_row,target_threshold,nett_pnl_sum,win_rate,target_1_pct,target_2_pct,target_3_pct,target_4_pct,total_trade,max_buy_zone_exceed_win,mean_buy_zone_exceed_win,median_buy_zone_exceed_win,max_buy_zone_exceed_lose,mean_buy_zone_exceed_lose,median_buy_zone_exceed_lose,tp_count,sl_count,sl_count_pct
i64,i64,i64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,i64,i64,f64
40,180,1500,-2.485497,63.636364,52.272727,50.757576,50.757576,50.0,264,-2.799956,-0.500572,-0.210392,-3.0,-2.854172,-3.0,2,88,33.333333


In [247]:
# Filter profitable rows
profitable_df = df_results.filter(pl.col("win_rate") >= 90)
# Get the row with the maximum opportunities
max_opportunities_profitable_params = profitable_df.sort("nett_pnl_sum", descending=True).row(0)
# Convert to DataFrame with explicit row orientation
max_opportunities_profitable_df = pl.DataFrame([max_opportunities_profitable_params], schema=df_results.schema, orient="row")
max_opportunities_profitable_df

prev_row,fast_reversal_row,target_threshold,nett_pnl_sum,win_rate,target_1_pct,target_2_pct,target_3_pct,target_4_pct,total_trade,max_buy_zone_exceed_win,mean_buy_zone_exceed_win,median_buy_zone_exceed_win,max_buy_zone_exceed_lose,mean_buy_zone_exceed_lose,median_buy_zone_exceed_lose,tp_count,sl_count,sl_count_pct
i64,i64,i64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,i64,i64,f64
300,20,1500,16.25969,100.0,33.333333,16.666667,16.666667,16.666667,6,-0.439435,-0.098889,0.0,null,null,null,0,0,0.0
